# LangChain应用开发

LangChain 是一套专为LLM 开发打造的开源框架，实现了 LLM 多种强大能力的利用，提供了 Chain、Agent、Tool 等多种封
装工具，基于 LangChain 可以便捷开发应用程序，极大化发挥 LLM 潜能。

In [1]:
# 功能代码，服务于下面的代码片段
import os
from openai import OpenAI

def get_completion(query_txt, temperature=0,is_stream=True):
    client = OpenAI(api_key='sk-8a2970d003e849e1a680a377eb16d22bxx',
                base_url = "https://dashscope.aliyuncs.com/compatible-mode/v1"
        )
    completion = client.chat.completions.create(
        model="qwen-plus",
        messages=[
            {"role": "user", "content": query_txt},
        ],
        temperature = temperature,
        stream=is_stream,
        # stream_options={"include_usage": True if is_stream else False}
    )

    if not is_stream:
        # print(completion.model_dump_json())
        print(completion.choices[0].message.content)
    else:
        for chunk in completion:
            # print(chunk.model_dump_json())
            if len(chunk.choices) > 0:
                delta = chunk.choices[0].delta
                if delta and delta.content:
                    # print(f"id: {chunk.id}, content: {chunk.choices[0].delta.content}")
                    print(chunk.choices[0].delta.content,end="")

def get_completion_from_messages(messages, model="qwen-max", temperature=0):
    client = OpenAI(api_key='sk-8a2970d003e849e1a680a377eb16d22bxx',
                base_url = "https://dashscope.aliyuncs.com/compatible-mode/v1"
        )
    completion = client.chat.completions.create(
    model=model,
    messages=messages,
    temperature=temperature, # 控制模型输出的随机程度
    )
    # print(str(response.choices[0].message))
    token_dict = {
        'prompt_tokens':completion.usage.prompt_tokens,
        'completion_tokens':completion.usage.completion_tokens,
        'total_tokens':completion.usage.total_tokens,
    }
    print("token_dict: %s" % token_dict)
    # print(inspect.getmembers(completion))
    return completion.choices[0].message.content

## 1. 通过Langchain使用openai

In [ ]:
#!pip3 install langchain_community
!pip3 list

In [68]:
from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0)
chat

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000190147A0E80>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000190147A1840>, temperature=0.0, model_kwargs={}, openai_api_key='sk-8a2970d003e849e1a680a377eb16d22b', openai_proxy='')

In [88]:
from langchain.chat_models import ChatOpenAI #HuggingFaceChatModel

# 假设 Qwen 提供了对话 API（需要自定义）
# class QwenChatAPI:
#     def __init__(self, api_key, api_url):
#         self.api_key = api_key
#         self.api_url = api_url

#     def predict(self, messages):
#         headers = {"Authorization": f"Bearer {self.api_key}"}
#         data = {"model": "qwen-7b", "messages": messages}
#         response = requests.post(self.api_url, headers=headers, json=data)
#         return response.json()["choices"][0]["message"]["content"]

chat = ChatOpenAI(
    model="qwen-plus",  # 替换为 Qwen 的对话模型名称（如果有）
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    api_key='sk-8a2970d003e849e1a680a377eb16d22bxx',
    base_url = "https://dashscope.aliyuncs.com/compatible-mode/v1"
)

# 测试调用
#print(chat([{"role": "user", "content": "Hello, how are you?"}]))
# messages = [
#     (
#         "system",
#         "You are a helpful assistant that translates English to French. Translate the user sentence.",
#     ),
#     ("human", "你是谁?请用中文回复"),
# ]
# ai_msg = chat.invoke(messages)
# ai_msg


1.1 测试邮件分析和回复

提示模版 prompt_template 需要两个输入变量： style 和 text 。 这里分别对应
* customer_style : 我们想要的顾客邮件风格
* customer_email : 顾客的原始邮件文本。

**使用提示模版，可以让我们更为方便地重复使用设计好的提示。**

| 提示词类型 | 定义 | 例子 |
| --- | --- | --- |
| PromptTemplate | prompt template就是一个prompt的模板，通过prompt template，我们可以快速的生成多个prompt。个其他型号的手机？ | 具体见下面示例|
| ChatPromptTemplate| chat消息就可以被分为AI, human或者system这几种角色 | 具体下面示例 |
| SystemMessagePromptTemplate | chat消息就可以被分为AI, human或者system这几种角色 | 具体下面示例
| HumanMessagePromptTemplate | chat消息就可以被分为AI, human或者system这几种角色 | 具体下面示例 |
| HumanMessagePromptTemplate | chat消息就可以被分为AI, human或者system这几种角色 | 具体下面示例 |

In [ ]:
from langchain.prompts import ChatPromptTemplate
# 首先，构造一个提示模版字符串：`template_string`
template_string = """把由三个反引号分隔的文本\
翻译成一种{style}风格。\
文本: ```{text}```
"""
# 然后，我们调用`ChatPromptTemplatee.from_template()`函数将
# 上面的提示模版字符`template_string`转换为提示模版`prompt_template`
prompt_template = ChatPromptTemplate.from_template(template_string)
print("\n", prompt_template.messages[0].prompt)

customer_style = """正式普通话 \
用一个平静、尊敬的语气
"""
customer_email = """
嗯呐，我现在可是火冒三丈，我那个搅拌机盖子竟然飞了出去，把我厨房的墙壁都溅上了果汁！
更糟糕的是，保修条款可不包括清理我厨房的费用。
伙计，赶紧给我过来！
"""
# 使用提示模版
customer_messages = prompt_template.format_messages(style=customer_style,text=customer_email)

# 打印客户消息类型
print("客户消息类型:",type(customer_messages),"\n")
# 打印第一个客户消息类型
print("第一个客户客户消息类型类型:", type(customer_messages[0]),"\n")

print("第一个客户客户消息类内容: ", customer_messages[0],"\n")

#提问对话
customer_response = chat(customer_messages)
print(customer_response.content)

In [ ]:
service_reply = """嘿，顾客， \
保修不包括厨房的清洁费用， \
因为您在启动搅拌机之前 \
忘记盖上盖子而误用搅拌机, \
这是您的错。 \
倒霉！ 再见！
"""
service_style_pirate = """\
一个有礼貌的语气 \
使用海盗风格\
"""
service_messages = prompt_template.format_messages(
style=service_style_pirate,
text=service_reply)
print("\n", service_messages[0].content)

# 调用模型部分定义的chat模型来转换回复消息风格
service_response = chat(service_messages)
print(service_response.content)


In [ ]:
from langchain.prompts import ChatPromptTemplate

# 客户评论示例
customer_review = """\
这款吹叶机非常神奇。它有四个设置：
吹蜡烛、微风、风城、龙卷风。
两天后就到了，正好赶上我妻子的周年纪念礼物。
我想我的妻子会喜欢它到说不出话来。
到目前为止，我是唯一一个使用它的人，而且我一直
每隔一天早上用它来清理草坪上的叶子。
它比其他吹叶机稍微贵一点，但我认为它的额外功能是值得的。
"""

# 提取信息的指令模板
review_template = """\
对于以下文本，请从中提取以下信息：

礼物：该商品是作为礼物送给别人的吗？
如果是，则回答 是的；如果否或未知，则回答 不是。

交货天数：产品需要多少天到达？
如果没有找到该信息，则输出-1。

价钱：提取有关价值或价格的任何句子，
并将它们输出为逗号分隔的 Python 列表。

使用以下键将输出格式化为 JSON：
礼物
交货天数
价钱

文本: {text}
"""

# 创建提示模板
prompt_template = ChatPromptTemplate.from_template(review_template)
print("提示模版：", prompt_template)

# 格式化消息并调用聊天模型
messages = prompt_template.format_messages(text=customer_review)
#chat = ChatOpenAI(temperature=0.0)
response = chat(messages)

# 输出结果
print("结果类型:", type(response.content))
print("结果:", response.content)
print("结果:", response)

使用输出解析器提取客户评价中的信息

In [31]:
review_template_2 = """\
对于以下文本，请从中提取以下信息：：
礼物：该商品是作为礼物送给别人的吗？
如果是，则回答 是的；如果否或未知，则回答 不是。
交货天数：产品到达需要多少天？ 如果没有找到该信息，则输出-1。
价钱：提取有关价值或价格的任何句子，并将它们输出为逗号分隔的 Python 列表。
文本: {text}
{format_instructions}
"""
prompt = ChatPromptTemplate.from_template(template=review_template_2)
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
gift_schema = ResponseSchema(name="礼物",description="这件物品是作为礼物送给别人的吗？\
如果是，则回答 是的，\
如果否或未知，则回答 不是。")
delivery_days_schema = ResponseSchema(name="交货天数",
description="产品需要多少天才能到达？\
如果没有找到该信息，则输出-1。")
price_value_schema = ResponseSchema(name="价钱",
description="提取有关价值或价格的任何句子，\
并将它们输出为逗号分隔的 Python 列表")
response_schemas = [gift_schema,
delivery_days_schema,
price_value_schema]
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print("输出解析器的数据格式规定：",format_instructions)

输出解析器的数据格式规定： The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"礼物": string  // 这件物品是作为礼物送给别人的吗？如果是，则回答 是的，如果否或未知，则回答 不是。
	"交货天数": string  // 产品需要多少天才能到达？如果没有找到该信息，则输出-1。
	"价钱": string  // 提取有关价值或价格的任何句子，并将它们输出为逗号分隔的 Python 列表
}
```


In [32]:
messages = prompt.format_messages(text=customer_review,
format_instructions=format_instructions)
print("第一条客户消息:",messages[0].content)

第一条客户消息: 对于以下文本，请从中提取以下信息：：
礼物：该商品是作为礼物送给别人的吗？
如果是，则回答 是的；如果否或未知，则回答 不是。
交货天数：产品到达需要多少天？ 如果没有找到该信息，则输出-1。
价钱：提取有关价值或价格的任何句子，并将它们输出为逗号分隔的 Python 列表。
文本: 这款吹叶机非常神奇。它有四个设置：
吹蜡烛、微风、风城、龙卷风。
两天后就到了，正好赶上我妻子的周年纪念礼物。
我想我的妻子会喜欢它到说不出话来。
到目前为止，我是唯一一个使用它的人，而且我一直
每隔一天早上用它来清理草坪上的叶子。
它比其他吹叶机稍微贵一点，但我认为它的额外功能是值得的。

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"礼物": string  // 这件物品是作为礼物送给别人的吗？如果是，则回答 是的，如果否或未知，则回答 不是。
	"交货天数": string  // 产品需要多少天才能到达？如果没有找到该信息，则输出-1。
	"价钱": string  // 提取有关价值或价格的任何句子，并将它们输出为逗号分隔的 Python 列表
}
```



In [ ]:
response = chat(messages)
print("结果类型:", type(response.content))
print("结果:", response.content)

output_dict = output_parser.parse(response.content)
print("解析后的结果类型:", type(output_dict))
print("解析后的结果:", output_dict)

## 2 存储
LangChain 中的储存模块，即如何将先前的对话嵌入到语言模型中的，使其具有连续对话的能力。当使用 LangChain 中的储存(Memory)模块时，它旨在保存、组织和跟踪整个对话的历史，从而为用户和模型之间的交互提供连续的上下文。

LangChain 提供了多种储存类型：
* 缓冲区储存允许保留最近的聊天消息，
* 摘要储存则提供了对整个对话的摘要。
* 实体储存则允许在多轮对话中保留有关特定实体的信息。

2.1 对话缓存存储

缓存存储，也称为对话历史，是 LLM 模型在处理用户输入时，将之前与用户交互的输入和输出进行存储，以供后续使用。

In [82]:
from langchain.chains import ConversationChain
#from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
# 这里我们将参数temperature设置为0.0，从而减少生成答案的随机性。
# 如果你想要每次得到不一样的有新意的答案，可以尝试增大该参数。
llm = chat #ChatOpenAI(temperature=0.0)
memory = ConversationBufferMemory()
# 新建一个 ConversationChain Class 实例
# verbose参数设置为True时，程序会输出更详细的信息，以提供更多的调试或运行时信息。
# 相反，当将verbose参数设置为False时，程序会以更简洁的方式运行，只输出关键的信息。
conversation = ConversationChain(llm=llm, memory = memory, verbose=True )

#run 1：
conversation.predict(input="你好, 我叫皮皮鲁")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: 你好, 我叫皮皮鲁
AI:

> Finished chain.


'你好，皮皮鲁！很高兴认识你。我是通义千问，阿里巴巴集团旗下的超大规模语言模型。你可以叫我Qwen。名字很特别啊，皮皮鲁是谁创造的呀？是不是郑渊洁笔下的那个充满想象力和冒险精神的小男孩呢？如果有什么问题或者想聊的话题，尽管告诉我哦！'

In [ ]:
conversation.predict(input="1+1等于多少？")


In [79]:
conversation.predict(input="我叫什么名字？")

'你叫**皮皮鲁**！我刚才已经知道了你的名字，如果有任何其他问题，我都乐意回答哦！ 🚀'

In [83]:
print(memory.buffer)

print(memory.load_memory_variables({}))

memory.save_context({"input": "很高兴和你成为朋友！"}, {"output": "是的，让我们一起去冒险吧！"})
print(memory.load_memory_variables({}))

Human: 你好, 我叫皮皮鲁
AI: 你好，皮皮鲁！很高兴认识你。我是通义千问，阿里巴巴集团旗下的超大规模语言模型。你可以叫我Qwen。名字很特别啊，皮皮鲁是谁创造的呀？是不是郑渊洁笔下的那个充满想象力和冒险精神的小男孩呢？如果有什么问题或者想聊的话题，尽管告诉我哦！
{'history': 'Human: 你好, 我叫皮皮鲁\nAI: 你好，皮皮鲁！很高兴认识你。我是通义千问，阿里巴巴集团旗下的超大规模语言模型。你可以叫我Qwen。名字很特别啊，皮皮鲁是谁创造的呀？是不是郑渊洁笔下的那个充满想象力和冒险精神的小男孩呢？如果有什么问题或者想聊的话题，尽管告诉我哦！'}
{'history': 'Human: 你好, 我叫皮皮鲁\nAI: 你好，皮皮鲁！很高兴认识你。我是通义千问，阿里巴巴集团旗下的超大规模语言模型。你可以叫我Qwen。名字很特别啊，皮皮鲁是谁创造的呀？是不是郑渊洁笔下的那个充满想象力和冒险精神的小男孩呢？如果有什么问题或者想聊的话题，尽管告诉我哦！\nHuman: 很高兴和你成为朋友！\nAI: 是的，让我们一起去冒险吧！'}


2.2 对话缓存窗口存储

保持缓存窗口存储的大小

In [77]:
from langchain.memory import ConversationBufferWindowMemory

#k参数控制着窗口大小，即窗口中存储的对话数量。
memory2 = ConversationBufferWindowMemory(k=1)
conversation2 = ConversationChain(llm=chat, memory=memory2, verbose=False )
print("第一轮对话：")
print(conversation2.predict(input="你好, 我叫皮皮鲁"))
print("第二轮对话：")
print(conversation2.predict(input="1+1等于多少？"))
print("第三轮对话：")
print(conversation2.predict(input="我叫什么名字？"))

第一轮对话：
你好，皮皮鲁！很高兴认识你。我是通义千问，一个超爱聊天和分享知识的AI。听说你是个充满想象力和冒险精神的人，是吗？要不要告诉我更多关于你的故事？或者我们可以聊聊有趣的话题，比如科幻、科技、文学……随便你挑！ 😊
第二轮对话：
1 + 1 等于 **2**。这是最基本的数学运算之一，皮皮鲁！如果你还有其他更复杂或者有趣的数学问题，也可以问我哦！ 😄
第三轮对话：
你在之前的提问中提到过你的名字是“皮皮鲁”。所以，我叫你皮皮鲁！如果有其他问题，尽管问吧！ 😊


2.3 对话字符缓存存储

使用对话字符缓存记忆，内存将限制保存的token数量。如果字符数量超出指定数目，它会切掉这个对话
的早期部分以保留与最近的交流相对应的字符数量，但不超过字符限制。

In [ ]:
!pip3 install tiktoken

In [ ]:
#from langchain.llms import OpenAI
from langchain.memory import ConversationTokenBufferMemory
memory = ConversationTokenBufferMemory(llm=chat, max_token_limit=30)
# memory.save_context({"input": "朝辞白帝彩云间，"}, {"output": "千里江陵一日还。"})
# memory.save_context({"input": "两岸猿声啼不住，"}, {"output": "轻舟已过万重山。"})
memory.save_context({"input": "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"}, {"output": "bbbbbbbbbbbbbbbbbbbbbbbbb"})
memory.save_context({"input": "ccccccccccccccccccc"}, {"output": "ddddddddddddddddddddddddddd"})
memory.load_memory_variables({})

2.4 对话摘要缓存存储

对话摘要缓存储存，使用 LLM 对到目前为止历史对话自动总结摘要，并将其保存下来

In [ ]:
# from langchain.chains import ConversationChain
# from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
# 创建一个长字符串
schedule = "在八点你和你的产品团队有一个会议。 \
你需要做一个PPT。 \
上午9点到12点你需要忙于LangChain。\
Langchain是一个有用的工具，因此你的项目进展的非常快。\
中午，在意大利餐厅与一位开车来的顾客共进午餐 \
走了一个多小时的路程与你见面，只为了解最新的 AI。 \
确保你带了笔记本电脑可以展示最新的 LLM 样例."
# chat = ChatOpenAI(temperature=0.0)
memory = ConversationSummaryBufferMemory(llm=chat, max_token_limit=100)
memory.save_context({"input": f"你好，我叫皮皮鲁"}, {"output": f"你好啊，我叫鲁西西"})
memory.save_context({"input": "很高兴和你成为朋友！"}, {"output": "是的，让我们一起去冒险吧！"})
memory.save_context({"input": "今天的日程安排是什么？"}, {"output": f"{schedule}"})
print(memory.load_memory_variables({})['history'])

## 3 模型链

链（Chains）通常将大语言模型（LLM）与提示（Prompt）结合在一起，基于此，我们可以对文本或数据进行一系列操作

### 3.1 大语言模型链 LLMChain

In [ ]:
import warnings
warnings.filterwarnings('ignore')
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
# 这里我们将参数temperature设置为0.0，从而减少生成答案的随机性。
# 如果你想要每次得到不一样的有新意的答案，可以尝试调整该参数。
llm = chat

prompt = ChatPromptTemplate.from_template("描述制造{product}的一个公司的最佳名称是什么? 直接给出名称")
chain = LLMChain(llm=llm, prompt=prompt)
product = "大号床单套装"
chain.run(product)

### 3.2 简单顺序链

顺序链（SequentialChains）是按预定义顺序执行其链接的链。具体来说，我们将使用简单顺序链（SimpleSequentialChain），这是顺序链的最简单类型，其中每个步骤都有一个输入/输出，一个步骤的输出是下一个步骤的输入。


In [100]:
from langchain.chains import SimpleSequentialChain

# 提示模板 1 ：这个提示将接受产品并返回最佳名称来描述该公司
first_prompt = ChatPromptTemplate.from_template(
"描述制造{product}的一个公司的最好的名称是什么"
) 
chain_one = LLMChain(llm=llm, prompt=first_prompt)
# 提示模板 2 ：接受公司名称，然后输出该公司的长为20个单词的描述
second_prompt = ChatPromptTemplate.from_template(
"写一个20字的描述对于下面这个\
公司：{company_name}的"
) 
chain_two = LLMChain(llm=llm, prompt=second_prompt)

#组装链
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],verbose=True)

#执行链
product = "大号床单套装"
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
为一家专注于制造大号床单套装的公司起一个好名字，需要结合品牌定位、目标客户群体以及产品的特点。以下是一些可能适合的名称建议：

### 1. **KingSize Linens**
   - 简洁明了，直接突出“大号”和“床单”的概念。
   - “KingSize”传达出高端和宽敞的感觉。

### 2. **Grand Beddings**
   - “Grand”意味着宏大、豪华，适合用来描述大尺寸的床单套装。
   - 给人一种优雅且舒适的品牌印象。

### 3. **Majestic Sheets**
   - “Majestic”有宏伟、壮丽的意思，能够吸引追求高品质睡眠体验的消费者。
   - 强调产品的奢华感和大气设计。

### 4. **Extraordinary Beds**
   - 名称中的“Extraordinary”（非凡）突出了与众不同，适合用于推广超大尺寸或定制款式的床单套装。

### 5. **Spacious Comfort**
   - 结合了“空间感”和“舒适度”，非常适合用来形容大号床单带来的宽敞性和柔软触感。

### 6. **Royal Dimensions**
   - “Royal”象征皇家般的品质，“Dimensions”则暗示产品覆盖各种大尺寸需求。
   - 让顾客联想到精致与尊贵。

### 7. **Giant Slumber Co.**
   - “Giant”代表巨大，而“Slumber”意指睡眠，整体听起来有趣又贴切。
   - 适合年轻化、轻松风格的品牌形象。

### 8. **Luxury Oversized Linens**
   - 如果你的品牌定位于高端市场，这个名称可以很好地传递出奢华和超大尺寸的特点。

### 9. **Big Dream Bedding**
   - 将“大号”与梦想结合，寓意人们可以在更大的床上享受更美好的梦境。
   - 富有情感共鸣，容易记住。

### 10. **Monarch Textiles**
   - “Monarch”意为君主，象征权威和领导地位。
   - 表达出公司在大号床单领域的专业性和领先地位。

选择具体名称时，还可以考虑以下几个因素

'KingSize Linens，专注高端大号床单，为追求舒适与品质的顾客带来皇家般睡眠体验。'

### 3.3 顺序链

当只有一个输入和一个输出时，简单顺序链（SimpleSequentialChain）即可实现。当有多个输入或多个输出时，我们则需要使用顺序链（SequentialChain）来实现

In [ ]:
import pandas as pd
from langchain.chains import SequentialChain
from langchain.chat_models import ChatOpenAI #导入OpenAI模型
from langchain.prompts import ChatPromptTemplate #导入聊天提示模板
from langchain.chains import LLMChain #导入LLM链。


#子链1
# prompt模板 1: 翻译成英语（把下面的review翻译成英语）
first_prompt = ChatPromptTemplate.from_template(
"把下面的评论review翻译成英文:"
"\n\n{Review}"
) 
#chain 1: 输入：Review 输出：英文的 Review
chain_one = LLMChain(llm=llm, prompt=first_prompt, output_key="English_Review")
#子链2
# prompt模板 2: 用一句话总结下面的 review
second_prompt = ChatPromptTemplate.from_template(
"请你用一句话来总结下面的评论review:"
"\n\n{English_Review}"
)
#chain 2: 输入：英文的Review 输出：总结
chain_two = LLMChain(llm=llm, prompt=second_prompt, output_key="summary")

#子链3
# prompt模板 3: 下面review使用的什么语言
third_prompt = ChatPromptTemplate.from_template(
"下面的评论review使用的什么语言:\n\n{Review}"
)
#chain 3: 输入：Review 输出：语言
chain_three = LLMChain(llm=llm, prompt=third_prompt, output_key="language")
#子链4
# prompt模板 4: 使用特定的语言对下面的总结写一个后续回复
fourth_prompt = ChatPromptTemplate.from_template(
"使用特定的语言对下面的总结写一个后续回复:"
"\n\n总结: {summary}\n\n语言: {language}"
) 
#chain 4: 输入： 总结, 语言 输出： 后续回复
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
output_key="followup_message")



#组合四个子链：
#输入：review
#输出：英文review，总结，后续回复
overall_chain = SequentialChain(
chains=[chain_one, chain_two, chain_three, chain_four],
input_variables=["Review"],
output_variables=["English_Review", "summary","followup_message"],
verbose=True
)

#测试运行链
review = """\
 Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre.
J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou
contrefaçon !?
"""

overall_chain(review)

### 3.4 路由链

一个路由链，它首先决定将它传递给哪个子链，然后将它传递给那个链。路由器由两个组件组成：
* 路由链（Router Chain）：路由器链本身，负责选择要调用的下一个链
* destination_chains：路由器链可以路由到的链


### 3.5 基于文档的问答

使用大语言模型构建一个能够回答关于给定文档和文档集合的问答系统是一种非常实用和有效的应用场
景。语言模型不仅利用了自己的通用知识，还可以充分运用外部输入文档的专业信息来回答用户问
题，显著提升答案的质量和适用性。构建这类基于外部文档的问答系统，可以让语言模型更好地服务于
具体场景，而不是停留在通用层面。

基于文档问答的这个过程，我们会涉及 LangChain 中的其他组件，比如：嵌入模型（Embedding
Models)和向量储存(Vector Stores)

具体应用如RAG(Retrieval Augmented Generation)


### 3.6 通过LLM进行评估实例

* 首先，我们使用 LLM 自动构建了问答测试集，包含问题及标准答案。
* 然后，同一 LLM 试图回答测试集中的所有问题，得到响应。
* 下一步，需要评估语言模型的回答是否正确。这里奇妙的是，我们再使用另一个 LLM 链进行判断，所以 LLM 既是“球员”，又是“裁判